In [ ]:
   
class mongo:
    

    def __init__(self,coll_name = "stocks_daily"):
        try:
            self.coll_name = coll_name
            self.client = pymongo.MongoClient('mongodb://localhost:27000')
            self.db = self.client[coll_name]
            self.stock_list = "cantrade.csv"
            self.ETF_list = "ETFList.csv"
            self.initiate_list = [self.stock_list,self.ETF_list]
            print ("Connection Successful")
        except:
            print ("Connection Failed")


    def get_all_quote(self):
        result = pd.DataFrame()
        all_stock_1 = pd.read_csv(directory + self.stock_list)
        all_stock_2 = pd.read_csv(directory + self.ETF_list)
        all_stock_2 = all_stock_2.rename(columns={"Symbol":"Ticker"})
        all_stock = all_stock_1.append(all_stock_2)
        all_stock = all_stock.reset_index()
        for i in range(99,len(all_stock),100):
            tic_list = all_stock.Ticker.iloc[i-99:i].astype(str)
            result = result.append(self.get_ondemand_quote(tic_list))   
        return result

    def query_database(self,stock, start_date = datetime.now()-timedelta(days =200),end_date=datetime.now()+timedelta(days = 1)):
        
        collection = self.db[stock]
        #self.delete_duplicates(stock)
        start_date=(start_date-datetime.utcfromtimestamp(0)).total_seconds()*1000

        end_date = (end_date-datetime.utcfromtimestamp(0)).total_seconds()*1000
        get = collection.find({"Ticker":stock,"TimeStamp":{"$gte":start_date,"$lte":end_date}}).sort("TimeStamp",pymongo.ASCENDING)
        get_frame = pd.DataFrame(list(get))
        if len(get_frame) == 0:
            print ("no data for " + stock)
            return
        if self.coll_name == "stocks_10minute": 
            get_frame[u'TimeStamp'] = get_frame[u'TimeStamp'].apply(lambda x: change_timezone(datetime.utcfromtimestamp(0) + timedelta(milliseconds= x), "US/Pacific"))
        else:
            get_frame[u'TimeStamp'] = get_frame[u'TimeStamp'].apply(lambda x: datetime.utcfromtimestamp(0) + timedelta(milliseconds= x))
        #get_frame = get_frame.rename(columns={"Close":"Adj_close"})
        get_frame = get_frame.reindex()
        try:    
            get_frame = get_frame.drop("Adj Close",axis =1)
        except:
            pass
     
        return get_frame

    def frame_to_mongo(self,data,collection_str):
        collection = self.db[collection_str]
        records = json.loads(data.T.to_json()).values()
        collection.insert_many(records)
        collection.find().close()

    def update_minute(self, i):     
        pre = time.time()
        try:
            print (i)
            collection = self.db[i]
            quote = da.get_quote_yahoo(i)
            #quote.loc[:,'TimeStamp']  = datetime.strptime(quote['TimeStamp'].values[0][0:10],"%Y-%m-%d")
            quote["regularMarketTime"] = datetime(datetime.utcnow().year,datetime.utcnow().month,datetime.utcnow().day,datetime.utcnow().hour,datetime.utcnow().minute)

            #quote["regularMarketTime"] = datetime.utcnow()
            quote["Ticker"] = quote.index[0]
            quote = quote.rename( {"regularMarketTime":"TimeStamp", "regularMarketDayHigh": "High","regularMarketDayLow": "Low", "regularMarketOpen":"Open", "regularMarketVolume":"Volume", "regularMarketPrice":"Close"},axis = "columns") 
            quote = json.loads(quote.T.to_json()).values()[0]
            collection.update_many({"TimeStamp":quote[u'TimeStamp']},{"$set": quote },upsert=True)
            #clear_output()
            print ("succeeded " + i)
        except Exception as e:
            print( e)
            print ("error occored in updating "+ str(i))
        post = time.time()
        return (post - pre)
    
    def update_db_multi(self):
        # Worker function defines in this file outside of the class
        pre = time.time()
        now_time = datetime(datetime.utcnow().year,datetime.utcnow().month,datetime.utcnow().day,datetime.utcnow().hour,datetime.utcnow().minute)
        jobs = []
        pool = multiprocessing.pool.ThreadPool(processes = 4)
        for j in self.initiate_list:
            all_stock = pd.read_csv(directory+j)
            for i in range(0,len(all_stock.Ticker)+1,500):
                tickers = all_stock.Ticker.iloc[i:i+499]
                pool.apply_async(update_worker,args=(self,tickers,now_time))
#                 multiprocessing.Process(target=update_worker, args=(self,tickers)).start()
        #         jobs.append(p)
        pool.close()
        pool.join()
        post = time.time()
        return post - pre
        
        
    
    
        
    def update_db(self,test = False):
        pre = time.time()
        now_time = datetime(datetime.utcnow().year,datetime.utcnow().month,datetime.utcnow().day,datetime.utcnow().hour,datetime.utcnow().minute)
        for j in self.initiate_list:
            all_stock = pd.read_csv(directory+j)
            for i in all_stock.Ticker:
                try:
                    print (i)
                    #client = pymongo.MongoClient()
                    #db = client["stocks_daily"]
                    collection = self.db[i]
                    quote = da.get_quote_yahoo(i)
                    #quote.loc[:,'TimeStamp']  = datetime.strptime(quote['TimeStamp'].values[0][0:10],"%Y-%m-%d")
                    if self.coll_name == "stocks_daily":
                    
                        quote["regularMarketTime"] = datetime(datetime.utcfromtimestamp(quote["regularMarketTime"]).year,datetime.utcfromtimestamp(quote["regularMarketTime"]).month,datetime.utcfromtimestamp(quote["regularMarketTime"]).day)
                        
                    else:
                         quote["regularMarketTime"] = now_time
                    quote["Ticker"] = quote.index[0]
                    quote = quote.rename( {"regularMarketTime":"TimeStamp", "regularMarketDayHigh": "High","regularMarketDayLow": "Low", "regularMarketOpen":"Open", "regularMarketVolume":"Volume", "regularMarketPrice":"Close"},axis = "columns") 
                    quote = json.loads(quote.T.to_json()).values()[0]
                    if test:
                        return quote
                    collection.update_many({"TimeStamp":quote[u'TimeStamp']},{"$set": quote },upsert=True)
                    clear_output()
                    print ("successed " + i)
                except Exception as e:
                    print ( e)
                    print ("error occored in updating "+ str(i))
                    continue 
        post = time.time()
        return post - pre

    def delete_duplicates(self,stock):

        collection = self.db[stock]
        cursor = collection.aggregate(
            [
                {"$group": {"_id": "$TimeStamp", "unique_id": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
                {"$match": {"count": { "$gte": 2 }}}
            ]
        )

        response = []
        for doc in cursor:
            del doc["unique_id"][0]
            for id in doc["unique_id"]:
                response.append(id)

        collection.delete_many({"_id": {"$in": response}})

    def initiate_db(self):
        for i in self.initiate_list:
            all_stock = pd.read_csv(directory+i)
            count = 0
            db = self.client["stocks_daily"]
            for i in tqdm(all_stock.Ticker):
                trial=0
                success = False
                while not success and trial<2:
                    try:
                        print ("getting "+str(i))
                        price_table = get_price_data([i],"realtimeday")
                        
                        this_collection = db[i]
                        self.frame_to_mongo(price_table,this_collection)
                        print ("\nfinished inserting db "+str(i))
                        
                        print ("Finished "+str(int(count)))
                        #print ("{0:0.2f}%".format( count/len(all_stock)*100))
                        self.delete_duplicates(i)
                        count +=1
                        success = True
                        if count/10 == count/10.0:
                            time.sleep(10)
                        clear_output()
                    except Exception as e:
                        print (e)
                        trial += 1
                        time.sleep(1)
                        print ("Tring again")
                        continue

    def translate_time(time, timezone = "US/Pacific"):
        return change_timezone(datetime.utcfromtimestamp(0) + timedelta(milliseconds= time), timezone)
    
    
    def clean_up(self,reduce=False):
        error = pd.DataFrame()
        
        for j in self.initiate_list:
            all_stock = pd.read_csv(directory+j)
            collection = self.db[all_stock.Ticker[0]]
            first = collection.find({"Ticker":all_stock.Ticker[0]}).sort("TimeStamp",pymongo.ASCENDING)
            first = pd.DataFrame(list(first))
            for i in all_stock.Ticker:
                collection = self.db[str(i)]
                delete_list = collection.find({"Ticker":str(i)}).sort("TimeStamp",pymongo.ASCENDING)
                delete_list = pd.DataFrame(list(delete_list))
                if not reduce:
                    if len(first) >= len(delete_list) - len(list(delete_list[:len(delete_list)/4])):
                        print ("Skipped: " + str(i))
                        continue
                try:
                    
                    delete_list[u'TimeStamp'] = delete_list[u'TimeStamp'].apply(lambda x: datetime.utcfromtimestamp(0) + timedelta(milliseconds= x))
                    collection.delete_many({"_id":{"$in":list(delete_list[:len(delete_list)/4]._id)}})
                    print ("Cleaned " + str(i))
                except Exception as e:
                    print (e)
                    error = error.append([(i,e)])
        return error
    
 

In [1]:
from my_libs import *

/Users/ken/anaconda3/envs/python2/lib/python2.7/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [3]:
client = py.MongoClient('mongodb://10.0.1.22:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false',tz_aware=True)

In [4]:
sqld = mongo()

Connection Successful


## MySQL to MongoDB Migration

In [5]:
sqld.conn.cursor.execute("use all_symbol")

table_names = sqld.conn.get_data("show tables")

for i in table_names.iloc[:,0]:
    if i in client["all_symbol"].list_collection_names(): continue
    print (i)
    content = sqld.conn.get_data("select * from `%s`"%i)
    ## cast datetime.date to datetime
    for col in content.columns:
        if "datetime.date" in str(type(content[col].iloc[0])):
            content[col] = content[col].apply(lambda x: datetime.combine(x,datetime.min.time()))
    db = client["all_symbol"][i]
    db.insert_many(content.T.to_dict().values())

screener


<ipython-input-5-8c524757caff>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  db.insert_many(content.T.to_dict().values())


screener_tos
screenerModel
screenermodel_tos
screenerVol
screenervol_tos
Stocks_info


In [3]:
class connect_mongo:
    def __init__(self, coll_name, table = None):
        self.client = pymongo.MongoClient('mongodb://10.0.1.22:27017',tz_aware=True)
        self.db = self.client[coll_name]
        if table is not None:
            self.table = self.db[table]
    
    def set_table_name(self,table):
        self.table = self.db[table]
    
    
    def delete_duplicates(self,key):

        cursor = self.table.aggregate(
            [
                {"$group": {"_id": "$%s"%key, "unique_id": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
                {"$match": {"count": { "$gte": 2 }}}
            ]
        )

        response = []
        
        for doc in cursor:
            del doc["unique_id"][0]
            for id in doc["unique_id"]:
                response.append(id)
        return response
    
    def frame_to_mongo(self,data):
        for col in data.columns:
            if "datetime.date" in str(type(data[col].iloc[0])):
                data[col] = data[col].apply(lambda x: datetime.combine(x,datetime.min.time()))
        self.table.insert_many(data.T.to_dict().values())
        
    def update_to_mongo(self,data,key):
        for col in data.columns:
            if "datetime.date" in str(type(data[col].iloc[0])):
                data[col] = data[col].apply(lambda x: datetime.combine(x,datetime.min.time()))
        data_index = list(data.to_dict()[key].values())
        data = list(data.T.to_dict().values())
        for i in range(len(data)):
            self.table.update_one({key:data_index[i]},{"$set":data[i]},upsert=True)
#         update = []
#         for i in range(len(data)):
#             update.append(pymongo.UpdateOne({key:data_index[i]},{"$set":data[i]},upsert=True))
#         self.table.bulk_wirte(update)

In [17]:
mongo_conn = connect_mongo("stocks_10minute")

In [18]:
mongo_conn.set_table_name("AAPL")

In [19]:
list(mongo_conn.table.find())

[{u'Close': 318.3,
  u'Fwd_Return': nan,
  u'High': 318.37,
  u'Low': 316.441,
  u'Open': 317.11,
  u'Return': nan,
  u'Ticker': u'AAPL',
  u'Time': 160000,
  u'TimeStamp': datetime.datetime(2020, 5, 27, 16, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x7fbd68aee250>),
  u'Volume': 25321791,
  u'_id': ObjectId('60298704797a11517aa96d9a')},
 {u'Close': 317.11,
  u'Fwd_Return': nan,
  u'High': 317.33,
  u'Low': 316.67,
  u'Open': 316.73,
  u'Return': -0.00375264103938696,
  u'Ticker': u'AAPL',
  u'Time': 153000,
  u'TimeStamp': datetime.datetime(2020, 5, 27, 15, 30, tzinfo=<bson.tz_util.FixedOffset object at 0x7fbd68aee250>),
  u'Volume': 22928387,
  u'_id': ObjectId('60298704797a11517aa96d9b')},
 {u'Close': 316.971,
  u'Fwd_Return': nan,
  u'High': 317.0,
  u'Low': 315.6222,
  u'Open': 316.02,
  u'Return': -0.000438525921929798,
  u'Ticker': u'AAPL',
  u'Time': 150000,
  u'TimeStamp': datetime.datetime(2020, 5, 27, 15, 0, tzinfo=<bson.tz_util.FixedOffset object at 0x7fbd68aee250>),
  

In [7]:
mongo_conn.update_to_mongo(data,"TimeStamp")

In [22]:
 def query_database(stock, backdays):
        start_date = datetime.now()-timedelta(days =backdays)
        mongo_conn = connect_mongo("stocks_10minute")
        end_date=datetime.now()+timedelta(days = 1)
        mongo_conn.set_table_name(stock)
        #self.delete_duplicates(stock)
#         start_date=(start_date-datetime.utcfromtimestamp(0)).total_seconds()*1000

#         end_date = (end_date-datetime.utcfromtimestamp(0)).total_seconds()*1000
        get = mongo_conn.table.find({"Ticker":stock,"TimeStamp":{"$gte":start_date,"$lte":end_date}}).sort("TimeStamp",pymongo.ASCENDING)
        get_frame = pd.DataFrame(list(get))
#         if len(get_frame) == 0:
#             print ("no data for " + stock)
#             return
#         if self.coll_name == "stocks_10minute": 
#             get_frame[u'TimeStamp'] = get_frame[u'TimeStamp'].apply(lambda x: change_timezone(datetime.utcfromtimestamp(0) + timedelta(milliseconds= x), "US/Pacific"))
#         else:
#             get_frame[u'TimeStamp'] = get_frame[u'TimeStamp'].apply(lambda x: datetime.utcfromtimestamp(0) + timedelta(milliseconds= x))
#         #get_frame = get_frame.rename(columns={"Close":"Adj_close"})
#         get_frame = get_frame.reindex()
#         try:    
#             get_frame = get_frame.drop("Adj Close",axis =1)
#         except:
#             pass
     
        return get_frame

In [24]:
query_database("AAPL",1)

""
